# ReAct but with LangGraph

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# libraries
import getpass
import os
import json

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    AnyMessage,
    SystemMessage,
    HumanMessage,
    ToolMessage,
    AIMessage,
)

# Test Connection

In [3]:
model = ChatOpenAI(model="gpt-3.5-turbo")
message = [
    SystemMessage(content="Translate the following from English to German"),
    HumanMessage("Hello world!"),
]
result = model.invoke(message)

In [4]:
result.content

'Hallo Welt!'

# Tools

In [5]:
import wikipedia

from langchain.tools import Tool
from langchain_core.tools import tool


# tools = [
#     Tool(
#         name="search_wikipedia_pages",
#         description=(
#             "Search for Wikipedia pages based on a query. "
#             "Input should be a search query string"
#         ),
#         func=lambda query: wikipedia.search(query),
#     ),
#     Tool(
#         name="get_wikipedia_page_summary",
#         description=(
#             "Get the summary of a Wikipedia page. "
#             "Input should be a page title as string "
#             "and how many sentences the result should be as a number"
#         ),
#         func=lambda x: wikipedia.summary(**eval(x) if isinstance(x, str) else x),
#     ),
#     Tool(
#         name="get_wikipedia_page_content",
#         description="Get the full content of a Wikipedia page. Input should be the title of the Wikipedia page.",
#         func=lambda title: wikipedia.page(title).content,
#     ),
# ]
@tool
def search_wikipedia_pages(query: str) -> str:
    """Search for Wikipedia pages based on a query. Input should be a search query string."""
    return wikipedia.search(query)


@tool
def get_wikipedia_page_summary(title: str, sentences: int = 2) -> str:
    """Get the summary of a Wikipedia page. Input should be a page title as string and how many sentences the result should be as a number."""
    return wikipedia.summary(title, sentences=sentences)


@tool
def get_wikipedia_page_content(title: str) -> str:
    """Get the full content of a Wikipedia page. Input should be the title of the Wikipedia page."""
    return wikipedia.page(title).content


tools = [search_wikipedia_pages, get_wikipedia_page_summary, get_wikipedia_page_content]

tool_names = [tool.name for tool in tools]

# Agent

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# model = ChatOpenAI(model="gpt-4o")
agent = create_react_agent(
    model=model,
    tools=tools,
    checkpointer=MemorySaver(),
    # state_modifier=prompt,
)


def ask_agent(content):
    input = {
        "messages": [HumanMessage(content=content)],
    }
    for s in agent.stream(
        input,
        stream_mode="values",
        config={
            "configurable": {"thread_id": "thread-1"},
        },
    ):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [9]:
question_1 = "Can give me a summary of AI in wikipedia?"
ask_agent(question_1)

================================ Human Message =================================

Can give me a summary of AI in wikipedia?
================================== Ai Message ==================================
Tool Calls:
  search_wikipedia_pages (call_RzQkoyt6ZbfnHY3ADy5VUwDJ)
 Call ID: call_RzQkoyt6ZbfnHY3ADy5VUwDJ
  Args:
    query: Artificial Intelligence
================================= Tool Message =================================
Name: search_wikipedia_pages

["Artificial intelligence", "Artificial general intelligence", "A.I. Artificial Intelligence", "Ethics of artificial intelligence", "Applications of artificial intelligence", "Artificial intelligence in healthcare", "History of artificial intelligence", "Hallucination (artificial intelligence)", "Generative artificial intelligence", "Artificial intelligence art"]
================================== Ai Message ==================================
Tool Calls:
  get_wikipedia_page_summary (call_xLqj18rD0OMf3Bk3ZNcAQZIX)
 Call ID: ca

In [10]:
question_2 = "Is there a page about Maria?"
ask_agent(question_2)

================================ Human Message =================================

Is there a page about Maria?
================================== Ai Message ==================================
Tool Calls:
  search_wikipedia_pages (call_AHqNH4wowfOQRK7zE7OYoU7k)
 Call ID: call_AHqNH4wowfOQRK7zE7OYoU7k
  Args:
    query: Maria
================================= Tool Message =================================
Name: search_wikipedia_pages

["Maria", "Mai Mizuhashi", "Maria Maria", "Maria Callas", "Maria Theresa", "The Marías", "Maria Christina", "Maria Rodriguez", "Maria (2024 film)", "Infanta Maria Josepha of Portugal"]
================================== Ai Message ==================================

There are multiple pages related to "Maria" on Wikipedia. Could you please specify which Maria you are referring to so I can provide more information about the specific page?


In [11]:
question_3 = "Can you show me what are the example?"
ask_agent(question_3)

================================ Human Message =================================

Can you show me what are the example?
================================== Ai Message ==================================
Tool Calls:
  get_wikipedia_page_summary (call_pIW059Zyy9mTM0ECtsTYGbxn)
 Call ID: call_pIW059Zyy9mTM0ECtsTYGbxn
  Args:
    title: Maria
    sentences: 2
  get_wikipedia_page_summary (call_LaZrtXgrjJjY0s4oftpGYVNP)
 Call ID: call_LaZrtXgrjJjY0s4oftpGYVNP
  Args:
    title: Mai Mizuhashi
    sentences: 2
  get_wikipedia_page_summary (call_cvK08xEaPn72Dfltr2JDhHqw)
 Call ID: call_cvK08xEaPn72Dfltr2JDhHqw
  Args:
    title: Maria Maria
    sentences: 2


d:\Projects\aisg-5-day-ai-agent\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\Projects\aisg-5-day-ai-agent\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


================================= Tool Message =================================
Name: get_wikipedia_page_summary

"Maria Maria" is a song by American rock band Santana featuring the Product G&B, included on Santana's 18th studio album, Supernatural (1999). The song was written by Wyclef Jean, Jerry "Wonda" Duplessis, Carlos Santana, Karl Perazzo, and Raul Rekow, while Jean and Duplessis produced it.
================================== Ai Message ==================================

Here are some examples of pages related to "Maria" on Wikipedia:

1. **Maria**: This seems to be a disambiguation page with multiple entries related to individuals, places, and other topics named Maria.
2. **Mai Mizuhashi**: Also known by her stage name MARiA, she is a Japanese pop singer, songwriter, dancer, model, and costume designer.
3. **Maria Maria**: A song by American rock band Santana featuring the Product G&B, included in Santana's 18th studio album, Supernatural (1999).
